In [6]:
from pathlib import Path
from perinuclear_analysis.image_loader import ImageLoader
from perinuclear_analysis.mip_creator import MIPCreator


In [7]:
nd2_path = Path("/Users/gabrielduarte/Documents/GitHub/SaezAtienzar/perinuclear_analysis/data/ALIX/nd2/anti_ALIX_15_min_1.nd2")
mip_out = nd2_path.with_name(nd2_path.stem + "_mip.tiff")
mip_out

PosixPath('/Users/gabrielduarte/Documents/GitHub/SaezAtienzar/perinuclear_analysis/data/ALIX/nd2/anti_ALIX_15_min_1_mip.tiff')

In [9]:
loader = ImageLoader()
zstack = loader.load_image(nd2_path)
zstack.shape

2025-08-29 15:23:20,820 - perinuclear_analysis.image_loader - INFO - Converting anti_ALIX_15_min_1.nd2 to TIFF format for pipeline compatibility
2025-08-29 15:23:20,822 - perinuclear_analysis.format_converter - INFO - Converting /Users/gabrielduarte/Documents/GitHub/SaezAtienzar/perinuclear_analysis/data/ALIX/nd2/anti_ALIX_15_min_1.nd2 to /Users/gabrielduarte/Documents/GitHub/SaezAtienzar/perinuclear_analysis/data/ALIX/nd2/anti_ALIX_15_min_1.tiff
/Users/gabrielduarte/miniforge3/envs/bioimage-coloc/lib/python3.12/site-packages/tifffile/tifffile.py:2451: UserWarning: <tifffile.TiffWriter 'anti_ALIX_15_min_1.tiff'> not writing description to ImageJ file
  warnings.warn(
2025-08-29 15:23:29,824 - perinuclear_analysis.format_converter - INFO - Metadata saved to /Users/gabrielduarte/Documents/GitHub/SaezAtienzar/perinuclear_analysis/data/ALIX/nd2/anti_ALIX_15_min_1.json
2025-08-29 15:23:29,824 - perinuclear_analysis.format_converter - INFO - Successfully converted to /Users/gabrielduarte/Doc

(23, 1800, 1800, 4)

In [10]:
zstack.dtype

dtype('uint16')

In [12]:
mip_creator = MIPCreator()
mip = mip_creator.create_mip(zstack, method="max")

mip_creator.save_mip_tiff(
    mip,
    mip_out,
    dtype="uint16",               # or 'auto'
    normalize=True,
    compress=6,
    metadata={"source": str(nd2_path), "projection": "max"},
    overwrite=True,
)

mip_out

2025-08-29 15:29:01,749 - perinuclear_analysis.mip_creator - INFO - Creating max projection from shape (23, 1800, 1800, 4)
2025-08-29 15:29:04,732 - perinuclear_analysis.mip_creator - INFO - Created max projection with shape (1800, 1800, 4)
2025-08-29 15:29:05,120 - perinuclear_analysis.mip_creator - INFO - Saved MIP to /Users/gabrielduarte/Documents/GitHub/SaezAtienzar/perinuclear_analysis/data/ALIX/nd2/anti_ALIX_15_min_1_mip.tiff (uint16, shape=(1800, 1800, 4))


PosixPath('/Users/gabrielduarte/Documents/GitHub/SaezAtienzar/perinuclear_analysis/data/ALIX/nd2/anti_ALIX_15_min_1_mip.tiff')

In [16]:
from pathlib import Path
import tifffile, json, numpy as np

mip_path = Path("/Users/gabrielduarte/Documents/GitHub/SaezAtienzar/perinuclear_analysis/data/ALIX/nd2/anti_ALIX_15_min_1_mip.tiff")

print(f"File: {mip_path.name}")
print(f"Size (MB): {mip_path.stat().st_size / 1e6:.2f}")

with tifffile.TiffFile(mip_path) as tif:
    s = tif.series[0]
    p0 = tif.pages[0]
    print("Pages:", len(tif.pages))
    print("Series shape:", s.shape, "dtype:", s.dtype, "axes:", s.axes)
    print("Photometric:", p0.photometric)
    print("Compression:", p0.compression)
    print("Resolution tags:", p0.tags.get("XResolution"), p0.tags.get("YResolution"), p0.tags.get("ResolutionUnit"))
    print("ImageJ metadata:", tif.imagej_metadata or {})
    desc = p0.description
    if desc and desc.strip().startswith("{"):
        md = json.loads(desc)
        print("Converted metadata keys:", list(md.keys())[:10])
        print("Pixel size (um):", md.get("pixel_size_um"))
        print("Channels:", md.get("dimensions", {}).get("channels"))

arr = tifffile.imread(str(mip_path))
print("Array:", arr.shape, arr.dtype, "min/max:", float(arr.min()), float(arr.max()))

File: anti_ALIX_15_min_1_mip.tiff
Size (MB): 4.97
Pages: 1800
Series shape: (1800, 1800, 4) dtype: uint16 axes: QYX
Photometric: 1
Compression: 32946
Resolution tags: TiffTag 282 XResolution @130 RATIONAL @368 = (1, 1) TiffTag 283 YResolution @142 RATIONAL @376 = (1, 1) TiffTag 296 ResolutionUnit @154 SHORT @162 = NONE
ImageJ metadata: {}
Converted metadata keys: ['source', 'projection', 'shape']
Pixel size (um): None
Channels: None
Array: (1800, 1800, 4) uint16 min/max: 0.0 65535.0
